In [1]:
import pandas as pd
import numpy as np
import datetime
import nltk
import math

In [2]:
comments_df = pd.read_csv('../data/comments_no_chinese_df.csv')
#no_chinese_df = pd.read_csv('../data/comments_no_chinese_df.csv')

In [3]:
print(comments_df.shape)
#print(no_chinese_df.shape)

(3029, 11)


##### Change tokens columns from str type to list type

In [17]:
from ast import literal_eval

def converter(x):
    return literal_eval(x)

comments_df = pd.read_csv('../data/comments_no_chinese_df.csv', converters={'tokens_new':converter, 'normalized_tokens':converter})
#no_chinese_df = pd.read_csv('../data/comments_no_chinese_df.csv', converters={'tokens_new':converter, 'normalized_tokens':converter})

In [18]:
comments_df.head(15)

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count
0,0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,"[Megathread, Supreme, Court, Ruling, on, Affir...","[megathread, supreme, court, ruling, affirmati...",62,78
1,1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,"[Thanks, to, everyone, who, engaged, in, insig...","[thank, engage, insightful, respectful, discou...",9,20
2,2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,"[I, would, prefer, using, a, process, that, ta...","[prefer, process, take, account, poverty, inst...",52,103
3,3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...","[u/Tungsten_,, Thanks, for, creating, a, secti...","[u/tungsten_,, thank, create, section, discuss...",126,269
4,4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,"[As, with, anything, related, to, Asians, in, ...","[relate, asians, politic, m, see, lot, non, as...",25,59
5,5,jq0mhrx,t3_14m8mf4,Pancake_muncher,2023-06-29 12:12:39,NaN,Yet colleges will allow alumni and doners in e...,"[Yet, colleges, will, allow, alumni, and, done...","[college, allow, alumnus, doner, easily, consi...",19,40
6,6,jq0mlbs,t3_14m8mf4,suberry,2023-06-29 12:13:16,NaN,I just hated Affirmative Action as a distracti...,"[I, just, hated, Affirmative, Action, as, a, d...","[hate, affirmative, action, distraction, banda...",78,171
7,7,jq0cvvn,t3_14m8mf4,Puzzled-Painter3301,2023-06-29 11:12:31,NaN,My own feeling is that I was never in love wit...,"[My, own, feeling, is, that, I, was, never, in...","[feeling, love, affirmative, action, possible,...",102,231
8,8,jq0jtzk,t3_14m8mf4,e9967780,2023-06-29 11:55:52,NaN,Anti Asian racism whether against East Asians ...,"[Anti, Asian, racism, whether, against, East, ...","[anti, asian, racism, east, asians, south, asi...",21,46
9,9,jq0kfzq,t3_14m8mf4,NaN,2023-06-29 11:59:41,NaN,Can we overturn legacy and athlete admissions ...,"[Can, we, overturn, legacy, and, athlete, admi...","[overturn, legacy, athlete, admission, point, ...",15,29


In [19]:
def str_to_datetime(str_):
    if isinstance(str_, str):
        return datetime.datetime.fromisoformat(str_)
    else:
        return None
        

comments_df['time_created_datetime'] = comments_df['time_created'].apply(lambda x: str_to_datetime(x))
#no_chinese_df['time_created_datetime'] = no_chinese_df['time_created'].apply(lambda x: str_to_datetime(x))

## Divide comments and posts created pre-SCOTUS and post-SCOTUS ruling

#### Impute rows without dates

In [20]:
no_datetime = comments_df[pd.isna(comments_df['time_created'])]
#no_datetime_no_chinese = no_chinese_df[pd.isna(no_chinese_df['time_created'])]

print(no_datetime.shape)
#print(no_datetime_no_chinese.shape)

(0, 12)


In [21]:
ruling_date = datetime.datetime(2023, 6, 29)
print(ruling_date)

2023-06-29 00:00:00


In [22]:
first_row = comments_df.iloc[0]
first_time_str = first_row['time_created']

print(type(first_time_str))

first_datetime = datetime.datetime.fromisoformat(first_time_str)
print(type(first_datetime))

print(isinstance(first_time_str, str))

<class 'str'>
<class 'datetime.datetime'>
True


In [23]:
comments_df.head(20)

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
0,0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,"[Megathread, Supreme, Court, Ruling, on, Affir...","[megathread, supreme, court, ruling, affirmati...",62,78,2023-06-29 10:54:44
1,1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,"[Thanks, to, everyone, who, engaged, in, insig...","[thank, engage, insightful, respectful, discou...",9,20,2023-06-30 11:33:11
2,2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,"[I, would, prefer, using, a, process, that, ta...","[prefer, process, take, account, poverty, inst...",52,103,2023-06-29 11:16:15
3,3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...","[u/Tungsten_,, Thanks, for, creating, a, secti...","[u/tungsten_,, thank, create, section, discuss...",126,269,2023-06-29 11:09:47
4,4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,"[As, with, anything, related, to, Asians, in, ...","[relate, asians, politic, m, see, lot, non, as...",25,59,2023-06-29 11:26:41
5,5,jq0mhrx,t3_14m8mf4,Pancake_muncher,2023-06-29 12:12:39,NaN,Yet colleges will allow alumni and doners in e...,"[Yet, colleges, will, allow, alumni, and, done...","[college, allow, alumnus, doner, easily, consi...",19,40,2023-06-29 12:12:39
6,6,jq0mlbs,t3_14m8mf4,suberry,2023-06-29 12:13:16,NaN,I just hated Affirmative Action as a distracti...,"[I, just, hated, Affirmative, Action, as, a, d...","[hate, affirmative, action, distraction, banda...",78,171,2023-06-29 12:13:16
7,7,jq0cvvn,t3_14m8mf4,Puzzled-Painter3301,2023-06-29 11:12:31,NaN,My own feeling is that I was never in love wit...,"[My, own, feeling, is, that, I, was, never, in...","[feeling, love, affirmative, action, possible,...",102,231,2023-06-29 11:12:31
8,8,jq0jtzk,t3_14m8mf4,e9967780,2023-06-29 11:55:52,NaN,Anti Asian racism whether against East Asians ...,"[Anti, Asian, racism, whether, against, East, ...","[anti, asian, racism, east, asians, south, asi...",21,46,2023-06-29 11:55:52
9,9,jq0kfzq,t3_14m8mf4,NaN,2023-06-29 11:59:41,NaN,Can we overturn legacy and athlete admissions ...,"[Can, we, overturn, legacy, and, athlete, admi...","[overturn, legacy, athlete, admission, point, ...",15,29,2023-06-29 11:59:41


In [24]:
first_datetime = comments_df.iloc[0]['time_created_datetime']
print(type(first_datetime))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [25]:
pre_scotus = comments_df[comments_df['time_created_datetime'] < datetime.datetime(2023, 6, 29)]

In [26]:
pre_scotus

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
337,362,9nk0do,NaN,unkle,2018-10-12 07:52:55,senyorito,"Anti-Asian Bias, Not Affirmative Action, Is on...","[Anti, Asian, Bias, Not, Affirmative, Action, ...","[anti, asian, bias, affirmative, action, trial...",8,13,2018-10-12 07:52:55
338,363,e7myklh,t3_9nk0do,InSearchOfGoodPun,2018-10-12 09:31:05,DOES NOT FOLD,This is a really excellent article. Since it's...,"[This, is, a, really, excellent, article, Sinc...","[excellent, article, paywall, m, copy, commenc...",586,1140,2018-10-12 09:31:05
339,364,e7n7i5t,t3_9nk0do,hank_scorpiooo,2018-10-12 11:39:07,NaN,This is exactly how I view this case. It's not...,"[This, is, exactly, how, I, view, this, case, ...","[exactly, view, case, affirmative, action, bla...",44,128,2018-10-12 11:39:07
340,365,e7mvknv,t3_9nk0do,Cr3X1eUZ,2018-10-12 08:43:45,NaN,"""We hate reverse discrimination!"" -- White peo...","[We, hate, reverse, discrimination, White, peo...","[hate, reverse, discrimination, white, people,...",12,14,2018-10-12 08:43:45
341,366,e7mwslu,t3_9nk0do,totpot,2018-10-12 09:03:39,NaN,What they really need to get rid of is legacy ...,"[What, they, really, need, to, get, rid, of, i...","[need, rid, legacy, admission]",4,11,2018-10-12 09:03:39
...,...,...,...,...,...,...,...,...,...,...,...,...
3024,3311,epcqlfa,t1_elt4c17,arientyse,2019-05-29 14:15:14,NaN,"Please don't refer to us as ""the blacks."" It's...","[Please, do, n't, refer, to, us, as, the, blac...","[refer, black, okay, black, people, harm]",6,20,2019-05-29 14:15:14
3025,3312,4ziaf0,NaN,unkle,2016-08-25 07:44:54,senyorito,From Self-Interest to Collective Morality: How...,"[From, Self, Interest, to, Collective, Moralit...","[self, interest, collective, morality, reframe...",11,20,2016-08-25 07:44:54
3026,3313,d6witvi,t3_4ziaf0,Swordbow,2016-08-25 14:47:22,NaN,The writer takes good nuggets but integrates t...,"[The, writer, takes, good, nuggets, but, integ...","[writer, take, good, nugget, integrate, questi...",156,354,2016-08-25 14:47:22
3027,3314,d6z9lhb,t3_4ziaf0,IRVCath,2016-08-27 17:02:04,"Fil-Am, 1.5 Generation","Interestingly enough, even though Filipinos ar...","[Interestingly, enough, even, though, Filipino...","[interestingly, filipino, relatively, disadvan...",51,101,2016-08-27 17:02:04


In [27]:
post_scotus = comments_df[comments_df['time_created_datetime'] >= datetime.datetime(2023, 6, 29)]

In [28]:
post_scotus

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
0,0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,"[Megathread, Supreme, Court, Ruling, on, Affir...","[megathread, supreme, court, ruling, affirmati...",62,78,2023-06-29 10:54:44
1,1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,"[Thanks, to, everyone, who, engaged, in, insig...","[thank, engage, insightful, respectful, discou...",9,20,2023-06-30 11:33:11
2,2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,"[I, would, prefer, using, a, process, that, ta...","[prefer, process, take, account, poverty, inst...",52,103,2023-06-29 11:16:15
3,3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...","[u/Tungsten_,, Thanks, for, creating, a, secti...","[u/tungsten_,, thank, create, section, discuss...",126,269,2023-06-29 11:09:47
4,4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,"[As, with, anything, related, to, Asians, in, ...","[relate, asians, politic, m, see, lot, non, as...",25,59,2023-06-29 11:26:41
...,...,...,...,...,...,...,...,...,...,...,...,...
334,358,jq24mg4,t1_jq1itxi,Different-Rip-2787,2023-06-29 18:03:45,NaN,I fail to see how this is 'detrimental' to any...,"[I, fail, to, see, how, this, is, detrimental,...","[fail, detrimental, black, brown, student, qua...",37,88,2023-06-29 18:03:45
335,359,jq3gway,t1_jq3f4ro,NaN,2023-06-30 00:57:44,NaN,>We have test scores to look at\n\nIf you beli...,"[>, We, have, test, scores, to, look, at, If, ...","[test, score, look, believe, well, test, score...",45,115,2023-06-30 00:57:44
336,361,jq2ectl,t1_jq25ucy,Different-Rip-2787,2023-06-29 19:16:06,NaN,> Those same options were also available to As...,"[>, Those, same, options, were, also, availabl...","[option, available, asam, student, difference,...",30,70,2023-06-29 19:16:06
3004,3290,1belws6,NaN,DudeGuyBroPalBuddie,2024-03-14 09:02:59,NaN,How White Supremacy uses Taiwan This is a long...,"[How, White, Supremacy, uses, Taiwan, This, is...","[white, supremacy, use, taiwan, long, read, se...",1235,2644,2024-03-14 09:02:59


No Chinese posts - Breakdown of datetimes:
- Before SCOTUS decision: 2690 rows
- After SCOTUS decision: 339 rows

- time_created is NaN: 0 rows

#### Look at the earliest June 29 posts/comments to see if they were truly created after SCOTUS decision was annoucned

In [30]:
sorted_post_scotus = sorted(post_scotus['time_created'])

In [31]:
print(sorted_post_scotus[:20])

['2023-06-29 10:54:44', '2023-06-29 11:02:09', '2023-06-29 11:04:36', '2023-06-29 11:09:47', '2023-06-29 11:10:11', '2023-06-29 11:11:53', '2023-06-29 11:12:31', '2023-06-29 11:13:09', '2023-06-29 11:16:07', '2023-06-29 11:16:15', '2023-06-29 11:19:14', '2023-06-29 11:21:11', '2023-06-29 11:21:18', '2023-06-29 11:22:58', '2023-06-29 11:24:49', '2023-06-29 11:26:41', '2023-06-29 11:28:19', '2023-06-29 11:31:15', '2023-06-29 11:32:11', '2023-06-29 11:32:45']


Earliest June 29 comments were from after SCOTUS decision was released - meaning the date June 29 can be used as the cutoff without regarding the exact time.

## Get pairs of same-user comments pre and post SCOTUS for comparison

### Get list of usernames that commented pre and post SCOTUS decision

In [32]:
pre_scotus_usernames = pre_scotus['username'].unique()
pre_scotus_usernames = pre_scotus_usernames[~pd.isnull(pre_scotus_usernames)] # remove nan
print(len(pre_scotus_usernames))
print(pre_scotus_usernames[:10])
post_scotus_usernames = post_scotus['username'].unique()
post_scotus_usernames = post_scotus_usernames[~pd.isnull(post_scotus_usernames)] # remove nan
print(len(post_scotus_usernames))
print(post_scotus_usernames[:10])

587
['unkle' 'InSearchOfGoodPun' 'hank_scorpiooo' 'Cr3X1eUZ' 'totpot'
 'Stoxastic' 'TangerineX' 'teknos1s' 'IceCreaaams' 'potatopotahto0']
124
['Tungsten_' 'ProudBlackMatt' 'TomatoCanned' 'bad-fengshui'
 'Pancake_muncher' 'suberry' 'Puzzled-Painter3301' 'e9967780'
 'OkartoIceCream' 'ShalomHasaeyo']


In [33]:
both_usernames = np.intersect1d(pre_scotus_usernames, post_scotus_usernames)

In [34]:
print(both_usernames)
print(len(both_usernames))

['Admiral_Wen' 'AwesomeAsian' 'Different-Rip-2787' 'Gryffinclaw'
 'Jazzlike_Ad_9118' 'JerichoMassey' 'MsNewKicks' 'OkartoIceCream'
 'Puzzled-Painter3301' 'bad-monkey' 'chilispicedmango' 'dropoutpanda'
 'kevintxu' 'littleglazed' 'suberry' 'thewhizzle' 'wildgift' 'xlsma']
18


There are 18 usernames which posted/commented pre and post SCOTUS decision.

In [35]:
matched_pre = pre_scotus[pre_scotus['username'].isin(both_usernames)]
matched_pre

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
389,424,hun0o3g,t1_hum0uby,Admiral_Wen,2022-01-28 15:46:09,NaN,This is my issue as well. Even as someone who ...,"[This, is, my, issue, as, well, Even, as, some...","[issue, believe, importance, diversity, think,...",126,254,2022-01-28 15:46:09
415,455,humca27,t1_hulefke,Different-Rip-2787,2022-01-28 13:10:44,NaN,"Also, it is completely bullshit that we have t...","[Also, it, is, completely, bullshit, that, we,...","[completely, bullshit, prove, typical, asian, ...",33,78,2022-01-28 13:10:44
441,486,hun0w2w,t1_humkls4,Different-Rip-2787,2022-01-28 15:47:39,NaN,My view is slightly different. I think we shou...,"[My, view, is, slightly, different, I, think, ...","[view, slightly, different, think, stereotype,...",20,63,2022-01-28 15:47:39
449,496,hunfs8l,t1_hun1nmu,Different-Rip-2787,2022-01-28 17:28:37,NaN,I see. And even there I don't think we should ...,"[I, see, And, even, there, I, do, n't, think, ...","[think, change, apologize, exist]",4,29,2022-01-28 17:28:37
464,511,fv0uk89,t3_h9tuc2,littleglazed,2020-06-16 11:02:28,1.5 gen Korean Am,i thought dismantling affirmative action worke...,"[i, thought, dismantling, affirmative, action,...","[think, dismantle, affirmative, action, work, ...",65,138,2020-06-16 11:02:28
...,...,...,...,...,...,...,...,...,...,...,...,...
2723,2999,cziu334,t3_43eu4h,MsNewKicks,2016-01-31 13:25:11,"First Of Her Name, Queen ABG, 나쁜 기집애, Blocker ...",I've never liked the idea of Affirmative Action.,"[I, 've, never, liked, the, idea, of, Affirmat...","[ve, like, idea, affirmative, action]",5,9,2016-01-31 13:25:11
2729,3005,i3e3k4v,t3_tuqh5o,Different-Rip-2787,2022-04-04 14:26:42,NaN,As a long time Democrat I have to say that Dem...,"[As, a, long, time, Democrat, I, have, to, say...","[long, time, democrat, democrats, risk, lose, ...",19,40,2022-04-04 14:26:42
2731,3007,i403o5d,t1_i3e3k4v,wildgift,2022-04-09 00:58:42,NaN,Polls still show we support affirmative action...,"[Polls, still, show, we, support, affirmative,...","[poll, support, affirmative, action, plank, ch...",34,80,2022-04-09 00:58:42
2743,3020,dwp6ki8,t3_88rs2g,bad-monkey,2018-04-02 18:08:14,NaN,"I want to add that ""Affirmative Action"" is mor...","[I, want, to, add, that, Affirmative, Action, ...","[want, add, affirmative, action, college, admi...",30,51,2018-04-02 18:08:14


In [36]:
matched_post = post_scotus[post_scotus['username'].isin(both_usernames)]
matched_post

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
6,6,jq0mlbs,t3_14m8mf4,suberry,2023-06-29 12:13:16,NaN,I just hated Affirmative Action as a distracti...,"[I, just, hated, Affirmative, Action, as, a, d...","[hate, affirmative, action, distraction, banda...",78,171,2023-06-29 12:13:16
7,7,jq0cvvn,t3_14m8mf4,Puzzled-Painter3301,2023-06-29 11:12:31,NaN,My own feeling is that I was never in love wit...,"[My, own, feeling, is, that, I, was, never, in...","[feeling, love, affirmative, action, possible,...",102,231,2023-06-29 11:12:31
10,10,jq0iyws,t3_14m8mf4,OkartoIceCream,2023-06-29 11:50:34,NaN,"I want to remind people that in California, on...","[I, want, to, remind, people, that, in, Califo...","[want, remind, people, california, progressive...",104,200,2023-06-29 11:50:34
13,13,jq2fp75,t3_14m8mf4,MsNewKicks,2023-06-29 19:26:21,"First Of Her Name, Queen ABG, 나쁜 기집애, Blocker ...",There is obviously a lot to it and no real eas...,"[There, is, obviously, a, lot, to, it, and, no...","[obviously, lot, real, easy, answer, satisfy, ...",46,147,2023-06-29 19:26:21
21,22,jq0y66y,t3_14m8mf4,xlsma,2023-06-29 13:25:44,NaN,"I always say, if an immigrant family of 3 or 4...","[I, always, say, if, an, immigrant, family, of...","[immigrant, family, come, united, states, doll...",40,101,2023-06-29 13:25:44
...,...,...,...,...,...,...,...,...,...,...,...,...
324,347,jq0km4m,t1_jq0k0wv,littleglazed,2023-06-29 12:00:46,1.5 gen Korean Am,i'm saying that this won't solve the symptom t...,"[i, 'm, saying, that, this, wo, n't, solve, th...","[m, say, will, solve, symptom, try, solve, asi...",48,102,2023-06-29 12:00:46
325,348,jq0l02y,t1_jq0kqou,littleglazed,2023-06-29 12:03:13,1.5 gen Korean Am,"cool, and now everyone else is still fucked. b...","[cool, and, now, everyone, else, is, still, fu...","[cool, fuck, fuck, right]",4,12,2023-06-29 12:03:13
328,352,jq3f4ro,t1_jq2r9vr,Different-Rip-2787,2023-06-30 00:38:00,NaN,It's not subjective. We have test scores to lo...,"[It, 's, not, subjective, We, have, test, scor...","[subjective, test, score, look, clear, asian, ...",21,41,2023-06-30 00:38:00
334,358,jq24mg4,t1_jq1itxi,Different-Rip-2787,2023-06-29 18:03:45,NaN,I fail to see how this is 'detrimental' to any...,"[I, fail, to, see, how, this, is, detrimental,...","[fail, detrimental, black, brown, student, qua...",37,88,2023-06-29 18:03:45


## Collocations

#### Overall

In [37]:
bgs_pre = nltk.collocations.BigramCollocationFinder.from_words(pre_scotus['normalized_tokens'].sum())

In [38]:
print(pre_scotus['normalized_tokens'].sum()[:10])

['anti', 'asian', 'bias', 'affirmative', 'action', 'trial', 'harvard', 'case', 'excellent', 'article']


In [39]:
def bigramScoring(count, wordsTuple, total):
    return count
bgs_pre.nbest(bigramScoring, 50)

[('affirmative', 'action'),
 ('asian', 'americans'),
 ('asian', 'american'),
 ('white', 'people'),
 ('high', 'school'),
 ('college', 'admission'),
 ('asian', 'student'),
 ('race', 'base'),
 ('test', 'score'),
 ('legacy', 'admission'),
 ('ivy', 'league'),
 ('support', 'affirmative'),
 ('high', 'education'),
 ('asian', 'applicant'),
 ('student', 'body'),
 ('white', 'student'),
 ('supreme', 'court'),
 ('black', 'hispanic'),
 ('black', 'people'),
 ('sit', 'score'),
 ('african', 'americans'),
 ('african', 'american'),
 ('model', 'minority'),
 ('asian', 'kid'),
 ('admission', 'officer'),
 ('feel', 'like'),
 ('asian', 'people'),
 ('holistic', 'admission'),
 ('black', 'latino'),
 ('chinese', 'americans'),
 ('east', 'asians'),
 ('admission', 'process'),
 ('american', 'student'),
 ('black', 'student'),
 ('middle', 'class'),
 ('american', 'applicant'),
 ('united', 'states'),
 ('white', 'applicant'),
 ('white', 'supremacy'),
 ('low', 'income'),
 ('admission', 'office'),
 ('m', 'sure'),
 ('harvard'

In [40]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
bgs_pre.score_ngrams(bigram_measures.likelihood_ratio)[:40]
# other options include student_t, chi_sq, likelihood_ratio, pmi

[(('affirmative', 'action'), 13173.493697834569),
 (('asian', 'americans'), 3006.8643657902894),
 (('asian', 'american'), 1939.4078486725957),
 (('ivy', 'league'), 1017.5749795397296),
 (('supreme', 'court'), 889.8526541624618),
 (('high', 'school'), 792.2989152952255),
 (('test', 'score'), 736.4655600383967),
 (('united', 'states'), 715.0105255432215),
 (('white', 'people'), 702.046471662481),
 (('student', 'body'), 601.2514954325783),
 (('college', 'admission'), 547.3724257978479),
 (('sit', 'score'), 490.862315788692),
 (('model', 'minority'), 479.9808117172274),
 (('high', 'education'), 478.568147028926),
 (('admission', 'officer'), 473.36079774968033),
 (('black', 'hispanic'), 457.8430099871589),
 (('race', 'base'), 438.3928690099989),
 (('civil', 'right'), 432.1593664218944),
 (('legacy', 'admission'), 427.0194435613008),
 (('edward', 'blum'), 423.5000472665354),
 (('personal', 'rating'), 418.9571380359112),
 (('middle', 'class'), 400.36168877778005),
 (('african', 'american'), 3

In [41]:
bgs_pre_t = bgs_pre.score_ngrams(bigram_measures.student_t)
bgs_pre_t[:40]

[(('affirmative', 'action'), 35.213733850270785),
 (('asian', 'americans'), 21.29600799214048),
 (('asian', 'american'), 17.927782214647493),
 (('white', 'people'), 12.57247175797453),
 (('high', 'school'), 11.428957962345413),
 (('college', 'admission'), 10.206275548292181),
 (('test', 'score'), 8.784393185162218),
 (('ivy', 'league'), 8.648403916627885),
 (('race', 'base'), 8.63806099826481),
 (('legacy', 'admission'), 8.525508948948586),
 (('high', 'education'), 8.343548192168367),
 (('asian', 'student'), 8.251887846628158),
 (('student', 'body'), 8.17110480780396),
 (('support', 'affirmative'), 8.168949529895889),
 (('supreme', 'court'), 7.9903420381645445),
 (('black', 'hispanic'), 7.832156531948366),
 (('asian', 'applicant'), 7.5628694152066425),
 (('sit', 'score'), 7.433577511477389),
 (('african', 'americans'), 7.30889740143293),
 (('model', 'minority'), 7.2927307004534265),
 (('african', 'american'), 7.255215238456539),
 (('admission', 'officer'), 7.150600280575593),
 (('holis

In [42]:
bgs_post = nltk.collocations.BigramCollocationFinder.from_words(post_scotus['normalized_tokens'].sum())

In [43]:
bgs_post_t = bgs_post.score_ngrams(bigram_measures.student_t)
bgs_post_t[:40]

[(('affirmative', 'action'), 10.970760096398934),
 (('asian', 'americans'), 7.919699261449686),
 (('legacy', 'admission'), 5.171609294355989),
 (('asian', 'american'), 4.618537107986249),
 (('supreme', 'court'), 4.345275267737646),
 (('black', 'student'), 4.106009461447),
 (('college', 'admission'), 3.6901859373800927),
 (('race', 'base'), 3.638878474763925),
 (('test', 'score'), 3.447129319160312),
 (('black', 'hispanic'), 3.420709030587105),
 (('white', 'people'), 3.3401004706251345),
 (('ivy', 'league'), 3.3074688153197194),
 (('white', 'student'), 3.3071859693014622),
 (('middle', 'class'), 3.2883175675367537),
 (('high', 'school'), 3.1733083477830086),
 (('student', 'body'), 3.1165039911270553),
 (('black', 'people'), 3.009441827908312),
 (('white', 'supremacy'), 2.96305356389709),
 (('western', 'country'), 2.961282159426402),
 (('base', 'affirmative'), 2.8361310276957687),
 (('united', 'states'), 2.8260412731015285),
 (('personality', 'score'), 2.8137541371315216),
 (('legacy', '

#### Chinese bi-grams

In [44]:
def find_chinese(n_gram_scores):
    chinese = []
    for i, elem in enumerate(n_gram_scores):
        if 'chinese' in elem[0]:
            chinese.append((i, elem[0], elem[1]))

    return chinese

In [45]:
chinese_pre = find_chinese(bgs_pre_t)
print(chinese_pre)

[(30, ('chinese', 'americans'), 6.589532753508237), (51, ('chinese', 'american'), 5.561507694202373), (135, ('chinese', 'immigrant'), 4.025049991137828), (221, ('chinese', 'people'), 3.4342337631327644), (428, ('chinese', 'kid'), 2.71170615296669), (496, ('bear', 'chinese'), 2.5732155501406457), (615, ('chinese', 'korean'), 2.4150168092031272), (760, ('han', 'chinese'), 2.229201939919727), (835, ('chinese', 'japanese'), 2.1902943936327075), (1187, ('chinese', 'restaurant'), 1.9795294309798832), (1359, ('chinese', 'indian'), 1.9347500612483781), (1438, ('chinese', 'woman'), 1.9027647971544457), (1449, ('chinese', 'ethnic'), 1.8976471548994165), (1478, ('class', 'chinese'), 1.8847557213199342), (1485, ('conservative', 'chinese'), 1.8822942281343291), (1953, ('estherwang', 'chinese'), 1.7276187993685415), (1992, ('chinese', 'community'), 1.7262061393559391), (2054, ('chinese', 'fob'), 1.7231867911682055), (2140, ('chinese', 'cop'), 1.7187547829678695), (2213, ('chinese', 'exclusion'), 1.7

In [46]:
chinese_post = find_chinese(bgs_post_t)
print(chinese_post)

[(207, ('war', 'chinese'), 1.7066288893472845), (3149, ('break', 'chinese'), 0.9984816533108393), (3151, ('camp', 'chinese'), 0.9984816533108393), (3157, ('chinese', 'billionaire'), 0.9984816533108393), (3158, ('chinese', 'brother'), 0.9984816533108393), (3159, ('chinese', 'luxury'), 0.9984816533108393), (3160, ('chinese', 'railroad'), 0.9984816533108393), (3161, ('chinese', 'superior'), 0.9984816533108393), (3191, ('kong', 'chinese'), 0.9984816533108393), (4084, ('chinese', 'decline'), 0.9969633066216786), (4085, ('chinese', 'indians'), 0.9969633066216786), (4093, ('desire', 'chinese'), 0.9969633066216786), (4138, ('series', 'chinese'), 0.9969633066216786), (4686, ('chinese', 'city'), 0.9954449599325179), (4687, ('chinese', 'excuse'), 0.9954449599325179), (5113, ('bomb', 'chinese'), 0.9939266132433572), (5116, ('chinese', 'koreans'), 0.9939266132433572), (5117, ('chinese', 'vietnamese'), 0.9939266132433572), (5449, ('offer', 'chinese'), 0.9924082665541966), (5459, ('shit', 'chinese'),

**Notes:**
- The bi-gram 'chinese' 'americans' disappears among the list of stat sig bigrams in post-SCOTUS user-matched/not-matched corpus.
- Explanation: In pre-SCOTUS corpus, there is a post about Chinese Americans polling most negatively toward affirmative action.

#### Matched users

In [47]:
bgs_pre_matched = nltk.collocations.BigramCollocationFinder.from_words(matched_pre['normalized_tokens'].sum())

In [48]:
print(matched_pre['normalized_tokens'].sum()[:10])

['issue', 'believe', 'importance', 'diversity', 'think', 'personality', 'score', 'step', 'backwards', 'racial']


In [49]:
def bigramScoring(count, wordsTuple, total):
    return count
bgs_pre_matched.nbest(bigramScoring, 50)

[('affirmative', 'action'),
 ('asian', 'americans'),
 ('black', 'brown'),
 ('high', 'education'),
 ('asian', 'american'),
 ('brown', 'student'),
 ('personality', 'score'),
 ('chinese', 'americans'),
 ('college', 'admission'),
 ('feel', 'like'),
 ('ivy', 'league'),
 ('support', 'affirmative'),
 ('asian', 'student'),
 ('sit', 'score'),
 ('small', 'business'),
 ('test', 'score'),
 ('white', 'applicant'),
 ('asian', 'group'),
 ('harvard', 'admission'),
 ('league', 'school'),
 ('student', 'body'),
 ('white', 'people'),
 ('white', 'student'),
 ('aa', 'asians'),
 ('action', 'workplace'),
 ('african', 'american'),
 ('asian', 'applicant'),
 ('asian', 'kid'),
 ('elite', 'school'),
 ('equal', 'footing'),
 ('help', 'asian'),
 ('less', 'school'),
 ('people', 'like'),
 ('safety', 'school'),
 ('aa', 'sport'),
 ('action', 'asian'),
 ('action', 'high'),
 ('admission', 'officer'),
 ('american', 'latino'),
 ('base', 'admission'),
 ('base', 'race'),
 ('business', 'owner'),
 ('forget', 'harvard'),
 ('go', 

In [50]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
bgs_pre_matched.score_ngrams(bigram_measures.likelihood_ratio)[:40]
# other options include student_t, chi_sq, likelihood_ratio, pmi

[(('affirmative', 'action'), 647.9458783991925),
 (('black', 'brown'), 126.28478319357154),
 (('high', 'education'), 118.51177445914368),
 (('asian', 'americans'), 110.62691436013851),
 (('ivy', 'league'), 79.93397764458607),
 (('personality', 'score'), 72.8436051757851),
 (('brown', 'student'), 69.91917723722574),
 (('equal', 'footing'), 56.21544988442817),
 (('feel', 'like'), 53.64640612193863),
 (('asian', 'american'), 51.79147600135923),
 (('small', 'business'), 50.65787230334619),
 (('student', 'body'), 47.17926141971135),
 (('chinese', 'americans'), 45.03900747087213),
 (('sit', 'score'), 44.92953570787786),
 (('test', 'score'), 44.92953570787786),
 (('african', 'american'), 43.40328610645263),
 (('graduation', 'rate'), 40.05629049133351),
 (('register', 'voter'), 40.05629049133351),
 (('self', 'report'), 40.05629049133351),
 (('white', 'applicant'), 39.92764651044131),
 (('college', 'admission'), 37.02377855091169),
 (('safety', 'school'), 36.85726018334951),
 (('league', 'schoo

In [51]:
bgs_pre_matched_t = bgs_pre_matched.score_ngrams(bigram_measures.student_t)

In [52]:
chinese_pre_matched = find_chinese(bgs_pre_matched_t)
print(chinese_pre_matched)

[(9, ('chinese', 'americans'), 2.5917187941526194), (149, ('chinese', 'immigrant'), 1.4033829641929343), (172, ('chinese', 'ethnic'), 1.3961625654061607), (185, ('bear', 'chinese'), 1.3925523660127737), (226, ('chinese', 'american'), 1.3383993751119707), (2056, ('apart', 'chinese'), 0.9948944070550012), (2058, ('chinese', 'exceptionalism'), 0.9948944070550012), (2059, ('chinese', 'object'), 0.9948944070550012), (2060, ('culturally', 'chinese'), 0.9948944070550012), (2062, ('exceptional', 'chinese'), 0.9948944070550012), (2065, ('gen', 'chinese'), 0.9948944070550012), (2070, ('https://muse.jhu.edu/article/794143', 'chinese'), 0.9948944070550012), (2072, ('loathe', 'chinese'), 0.9948944070550012), (2078, ('pattern', 'chinese'), 0.9948944070550012), (2080, ('pre-1965', 'chinese'), 0.9948944070550012), (2083, ('spearhead', 'chinese'), 0.9948944070550012), (2597, ('chinese', 'city'), 0.9897888141100023), (2598, ('chinese', 'figure'), 0.9897888141100023), (2600, ('figure', 'chinese'), 0.9897

In [53]:
bgs_post_matched = nltk.collocations.BigramCollocationFinder.from_words(matched_post['normalized_tokens'].sum())

In [54]:
def bigramScoring(count, wordsTuple, total):
    return count
bgs_post_matched.nbest(bigramScoring, 50)

[('affirmative', 'action'),
 ('black', 'student'),
 ('asian', 'americans'),
 ('black', 'brown'),
 ('race', 'base'),
 ('student', 'body'),
 ('test', 'score'),
 ('asian', 'american'),
 ('community', 'college'),
 ('ivy', 'league'),
 ('school', 'student'),
 ('white', 'student'),
 ('admit', 'class'),
 ('asian', 'student'),
 ('base', 'affirmative'),
 ('brown', 'student'),
 ('diversity', 'campus'),
 ('diversity', 'important'),
 ('high', 'school'),
 ('immigrant', 'family'),
 ('legacy', 'admission'),
 ('people', 'take'),
 ('program', 'help'),
 ('public', 'school'),
 ('public', 'university'),
 ('socioeconomic', 'class'),
 ('stop', 'obsess'),
 ('student', 'actually'),
 ('student', 'admit'),
 ('student', 'population'),
 ('aa', 'legacy'),
 ('action', 'program'),
 ('action', 'year'),
 ('actually', 'fairly'),
 ('admission', 'policy'),
 ('admit', 'student'),
 ('asian', 'applicant'),
 ('asian', 'parent'),
 ('asians', 'win'),
 ('bad', 'result'),
 ('base', 'admission'),
 ('base', 'system'),
 ('basically'

In [57]:
bgs_post_matched.score_ngrams(bigram_measures.likelihood_ratio)[:40]
# other options include student_t, chi_sq, likelihood_ratio, pmi

[(('affirmative', 'action'), 311.0139608054739),
 (('asian', 'americans'), 85.26004382022377),
 (('black', 'student'), 61.62313456601728),
 (('test', 'score'), 55.95372077937868),
 (('race', 'base'), 47.05137620441582),
 (('ivy', 'league'), 46.55014523663702),
 (('black', 'brown'), 45.41063559496736),
 (('student', 'body'), 38.66016412822315),
 (('stop', 'obsess'), 38.227865500628994),
 (('dynamic', 'socially'), 31.594857967611805),
 (('financial', 'aid'), 31.594857967611805),
 (('pay', 'wish'), 31.594857967611805),
 (('qualify', 'financial'), 31.594857967611805),
 (('select', 'pay'), 31.594857967611805),
 (('skin', 'color'), 31.594857967611805),
 (('socially', 'challenging'), 31.594857967611805),
 (('sociology', 'stem'), 31.594857967611805),
 (('son', 'daughter'), 31.594857967611805),
 (('immigrant', 'family'), 30.902633061440138),
 (('asian', 'american'), 28.29002196704477),
 (('vote', 'republican'), 27.775772957843806),
 (('basically', 'disappear'), 26.049680523132245),
 (('campus',

In [58]:
bgs_post_matched_t = bgs_post_matched.score_ngrams(bigram_measures.student_t)

In [59]:
chinese_post_matched = find_chinese(bgs_post_matched_t)
print(chinese_post_matched)

[(318, ('break', 'chinese'), 0.9989914271306102), (322, ('chinese', 'koreans'), 0.9989914271306102), (537, ('chinese', 'vietnamese'), 0.9979828542612204), (583, ('poc', 'chinese'), 0.9979828542612204)]


From the 18 users which posted/commented pre and post SCOTUS decision, the pre-SCOTUS comments still show that Chinese American collocations are still numerous and statistically significant. The post-SCOTUS comments show that Chinese American collocations are few and not stat sig.

### Next: Find differences in stat sig collocations between pre-SCOTUS and post-SCOTUS corpora